<a href="https://colab.research.google.com/github/OCCI-Staar/Curve-Maps-With-Clouds/blob/main/Curve_Maps_SHP_Clouds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Note this will **NOT** filter out clouds
## Create a standard environment

*   Simply Click play button to run all scripts

*   If no errors, continue to 1)

In [1]:
#Just need the required versions from GitHub
!wget -O required-info.txt https://raw.githubusercontent.com/OCCI-Staar/Curve-Maps-With-Clouds/main/required-info.txt

--2024-07-09 16:14:35--  https://raw.githubusercontent.com/OCCI-Staar/Curve-Maps-With-Clouds/main/required-info.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128 [text/plain]
Saving to: ‘required-info.txt’

required-info.txt   100%[===================>]     128  --.-KB/s    in 0s      

2024-07-09 16:14:35 (9.48 MB/s) - ‘required-info.txt’ saved [128/128]



In [2]:
# View the content of required-info.txt
!cat required-info.txt

# Install packages listed in required-info.txt
!pip install -r required-info.txt

pandas==2.0.3 
geopandas==0.14.4 
shapely==2.0.2 
fiona==1.9.6
pyproj==3.6.1 
geetools==0.6.11 
earthengine-api==0.1.347


In [3]:
#Needed Libraries
import ee
import time
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal
from datetime import datetime
from geetools import batch
import ipywidgets as widgets

from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt

# 1) Authenticate yourself through Google Earth Engine

In [4]:
#Authenticate the GEE User
ee.Authenticate()
# Verify if GEE server was authenticated
try:
  ee.Initialize()
except ee.EEException as error:
  print(f'ERROR: {error}')

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=tW8uO-rmKQWWd9plRynLIDrUvfmmj2KbLP2ouOsXXAo&tc=k1nBLZYi6ODrOKn3yL26ISnSS3kAK_blc1lrYdSb2iU&cc=QZX1JWztOgV-evHOaSpV-S9wvp9O_I105Yhnc1eRaO0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ATx3LY705um0BPZxV19a5TOudtZCZcvwtZSodB87PlSUPETWE1CHeKQd5SY

Successfully saved authorization token.


# 2) Supply Satellite, Date Range, Field Name

In [5]:
satellite = "COPERNICUS/S2_SR_HARMONIZED" # @param ["LANDSAT/LT04/C01/T1_SR", "LANDSAT/LT05/C01/T1_SR", "LANDSAT/LE07/C01/T1_SR", "LANDSAT/LC08/C01/T1_SR", "COPERNICUS/S2_SR_HARMONIZED"]
first_date = '2022-04-01'  #@param {type: "date"}
last_date  = '2022-11-30'  #@param {type: "date"}
field_name = "J7" #@param {type:"string"}


if satellite == "COPERNICUS/S2_SR_HARMONIZED":
      sat = 'S2SR'
if satellite == "LANDSAT/LT04/C01/T1_SR":
      sat = 'LT04'
if satellite == "LANDSAT/LT05/C01/T1_SR":
      sat = 'LT05'
if satellite == "LANDSAT/LE07/C01/T1_SR":
      sat = 'LE07'
if satellite == "LANDSAT/LC08/C01/T1_SR":
      sat = 'LC08'

# 3) Choose Field Boundaries file you want processed.


*   File extension must be **.shp**


In [6]:
from google.colab import files
shp_file = files.upload()

Saving Gatez_J7_Boundary_poly.shp to Gatez_J7_Boundary_poly.shp



*   Now choose **.shx** extension

In [7]:
shx_file = files.upload()

Saving Gatez_J7_Boundary_poly.shx to Gatez_J7_Boundary_poly.shx


*   Now choose **.dbf** extension

In [8]:
dbf_file = files.upload()

Saving Gatez_J7_Boundary_poly.dbf to Gatez_J7_Boundary_poly.dbf


*   Now choose **.prj** extension

In [9]:
#prj_file = files.upload()

Saving Gatez_J7_Boundary_poly.prj to Gatez_J7_Boundary_poly.prj


All files chosen?

In [19]:
shp_path = next(iter(shp_file))
shx_path = next(iter(shx_file))
dbf_path = next(iter(dbf_file))
#prj_path = next(iter(prj_file)) this causes an error and isn't needed

base_filename = shp_path.split('.shp')[0]
bound_files = []
bound_files = [shp_path, shx_path, dbf_path]
print(bound_files)

['Gatez_J7_Boundary_poly.shp', 'Gatez_J7_Boundary_poly.shx', 'Gatez_J7_Boundary_poly.dbf']


# 4) Time to process and export images

In [20]:
#With Clouds
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import satellite info
    s2_sr_col = (ee.ImageCollection(satellite)
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    return s2_sr_col

In [21]:
#Pulls geometry from the polygon
def get_bbox_coords(pol):
    shp_bb = base_filename  + '_bb.geojson'
    #Convert polygon CRS to WGS84 DD
    pol = pol.to_crs('EPSG:4326')
    #Create a buffer for the boundary and save as geoJSON
    pol.envelope.to_file(filename=shp_bb, driver="GeoJSON")
    #Open GeoJSON
    geom = json.loads(pol.envelope.to_json())['features'][0]['geometry']

    return geom

In [22]:
for bound_file in bound_files:

  pol = gpd.read_file(bound_file)

  pol = pol.dissolve()

  pol = pol.buffer(0.005)

  geom = get_bbox_coords(pol)

  #Get coordinates from GeoJSON
  coords = geom['coordinates']
  regions = ee.Geometry.Polygon(coords)

  start_date = ee.Date(first_date)
  end_date = ee.Date(last_date)

  if sat == 'S2SR':
    scale = 10
    extra = dict(sat=sat)
    s2_sr_cld_col = get_s2_sr_cld_col(regions, start_date, end_date)
    imagery = s2_sr_cld_col

field_name_sat = f'{field_name}'+"{id}"

print(imagery.size().getInfo())
batch.Export.imagecollection.toDrive(imagery, folder=f'YieldCurve_{sat}_{field_name}',
                                     region=regions, scale=scale, verbose = True,
                                     namePattern = field_name_sat)
time.sleep(300)
print("Export tasks were a success!")

<ipython-input-22-036f14e5c1ca>:7: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pol = pol.buffer(0.005)
<ipython-input-22-036f14e5c1ca>:7: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pol = pol.buffer(0.005)
<ipython-input-22-036f14e5c1ca>:7: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pol = pol.buffer(0.005)


48
exporting J720220401T183931_20220401T184945_T11UQT to folder 'YieldCurve_S2SR_J7' in GDrive
exporting J720220406T183919_20220406T184519_T11UQT to folder 'YieldCurve_S2SR_J7' in GDrive
exporting J720220411T183921_20220411T183924_T11UQT to folder 'YieldCurve_S2SR_J7' in GDrive
exporting J720220416T183919_20220416T184757_T11UQT to folder 'YieldCurve_S2SR_J7' in GDrive
exporting J720220421T183921_20220421T184650_T11UQT to folder 'YieldCurve_S2SR_J7' in GDrive
exporting J720220426T183909_20220426T184646_T11UQT to folder 'YieldCurve_S2SR_J7' in GDrive
exporting J720220501T183921_20220501T184126_T11UQT to folder 'YieldCurve_S2SR_J7' in GDrive
exporting J720220506T183919_20220506T184457_T11UQT to folder 'YieldCurve_S2SR_J7' in GDrive
exporting J720220511T183921_20220511T183922_T11UQT to folder 'YieldCurve_S2SR_J7' in GDrive
exporting J720220516T183919_20220516T184710_T11UQT to folder 'YieldCurve_S2SR_J7' in GDrive
exporting J720220521T183931_20220521T184251_T11UQT to folder 'YieldCurve_S2SR